In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [5]:
import pandas as pd
import numpy as np
import datetime
pd.set_option('display.max_colwidth', None) #显示单元格完整信息
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

user = pd.read_csv(r'C:\Users\User\Desktop\SEO_0807\会员列表导出.csv',encoding='gbk')
firChargeUser = pd.read_csv(r'C:\Users\User\Desktop\SEO_0807\会员首存报表.csv',encoding='gbk')
data = pd.read_excel(r'C:\Users\User\Desktop\SEO_0807\data\今日数据.xlsx')
daili = pd.read_excel(r'C:\Users\User\Desktop\SEO_0807\SEO每日模板-每日更新.xlsx','代理总表')

shuju = pd.DataFrame({'人员':['Paddy', 'Tony', 'Max', 'Martin', 'Zed', 'Hugo', 'Aber', 'DK', 'Ben','当日汇总'],
                      '日期':(datetime.datetime.now()+datetime.timedelta(days=-3)).strftime('%Y/%m/%d'),
                      '发送IP':[i for i in range(10)],'接受IP':[i for i in range(10)]})
shuju.set_index('人员',inplace = True)
shuju.sort_index(inplace=True)

data['IP']=pd.to_numeric(data['IP'],errors='coerce').replace(np.nan,0)
grp=data.groupby('网站名(domain)').agg({'IP':sum})

shuju.loc['Paddy','发送IP']=grp.loc['paddy.com','IP']
shuju.loc['Paddy','接受IP']=grp.loc['paddy.bty','IP']
shuju.loc['Tony','发送IP']=grp.loc['tonyb.com','IP']/2
shuju.loc['Tony','接受IP']=grp.loc['tonyb.com','IP']
shuju.loc['Max','发送IP']=grp.loc['mulu.com','IP']
shuju.loc['Max','接受IP']=grp.loc['max.bty','IP']
shuju.loc['Martin','发送IP']=grp.loc['redquan.com','IP']
shuju.loc['Martin','接受IP']=grp.loc['martin.bty','IP']
shuju.loc['Zed','发送IP']=grp.loc['zed.com','IP']
shuju.loc['Zed','接受IP']=grp.loc['zed.bty','IP']
shuju.loc['Hugo','发送IP']=grp.loc['hugo.com','IP']
# shuju.loc['Hugo','接受IP']=grp.loc['hugo.bty','IP']
shuju.loc['Hugo','接受IP']=0

shuju.loc['Aber','发送IP']=grp.loc['aber.com','IP']/2
shuju.loc['Aber','接受IP']=grp.loc['aber.bty','IP']
shuju.loc['DK','发送IP']=grp.loc['dk.com','IP']/2
shuju.loc['DK','接受IP']=grp.loc['dk.bty','IP']
shuju.loc['Ben','发送IP']=grp.loc['ben.com','IP']/2
shuju.loc['Ben','接受IP']=grp.loc['ben.bty','IP']

shuju['日期'] = pd.to_datetime(shuju['日期'])
shuju.insert(1,'人员2',shuju.index)
shuju['人员2']=shuju['人员2'].str.lower()
shuju.reset_index(inplace=True)

merge_user = pd.merge(user,daili,how = 'left',left_on='代理',right_on='代理线')
grpSEO = merge_user.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
grpSEO.rename(columns={'seo变化数据团队':'注册'},inplace=True)
grpSEO.reset_index(inplace=True)
grpSEO['人员2'] = grpSEO['seo变化数据团队'].str.lower()
grpSEO.set_index('seo变化数据团队',inplace=True)

# 第1次merge
shuju = pd.merge(shuju,grpSEO,on='人员2')
shuju['注册率'] = shuju['注册']/shuju['发送IP']*100

merge_charge = pd.merge(firChargeUser,daili,how='left',left_on='所属代理',right_on='代理线')
grpCHARGE = merge_charge.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
grpCHARGE= grpCHARGE.rename(columns={'seo变化数据团队':'开户'})
grpCHARGE.reset_index(inplace=True)
grpCHARGE['seo变化数据团队']=grpCHARGE['seo变化数据团队'].str.lower()
grpCHARGE= grpCHARGE.rename(columns={'seo变化数据团队':'人员2'})

# 第2次merge
shuju = pd.merge(shuju,grpCHARGE,how='left',on='人员2')
shuju['转化率'] = shuju['开户']/shuju['注册']*100

grp3  = merge_charge[merge_charge['注册时间'].str[:9]==merge_charge['交易时间'].str[:9]].groupby('seo变化数据团队').agg({'seo变化数据团队':len})
grp3.rename(columns = {'seo变化数据团队':'当日注册并开户'},inplace=True)
grp3.reset_index(inplace=True)
grp3['seo变化数据团队'] =grp3['seo变化数据团队'].str.lower()
grp3.rename(columns = {'seo变化数据团队':'人员2'},inplace=True)
# 第3次merge
shuju  = pd.merge(shuju,grp3,how='left',on='人员2')
shuju['当日注册激活率'] = shuju['当日注册并开户']/shuju['注册']*100

his_data  = pd.read_excel(r'C:\Users\User\Desktop\SEO_0807\SEO每日模板-每日更新.xlsx','历史')
be_data = his_data[his_data['日期']==(shuju['日期'][0]+datetime.timedelta(days=-2))]

shuju.set_index('人员',inplace=True)
shuju.sort_index(inplace=True)
be_data.set_index('人员',inplace=True)
be_data.sort_index(inplace=True)

be3_data = his_data[his_data['日期']>(shuju['日期'][0]+datetime.timedelta(days=-3))]
be3_data = be3_data.groupby('人员').mean()
be5_data = his_data[his_data['日期']>(shuju['日期'][0]+datetime.timedelta(days=-5))]
be5_data = be5_data.groupby('人员').mean()
be7_data = his_data[his_data['日期']>(shuju['日期'][0]+datetime.timedelta(days=-7))]
be7_data = be7_data.groupby('人员').mean()

all_ip= shuju.iloc[:,:3]
all_ip['对比昨天(总IP)']=shuju['发送IP']-be_data['总IP']
all_ip['对比前3天均值(总IP)']= shuju['发送IP']-be3_data['总IP']
all_ip['对比前5天均值(总IP)']= shuju['发送IP']-be5_data['总IP']
all_ip['对比前7天均值(总IP)']= shuju['发送IP']-be7_data['总IP']

all_ip['对比昨天(总注册)']=shuju['发送IP']-be_data['总IP']
all_ip['对比前3天均值(总注册)']= shuju['注册']-be3_data['注册']
all_ip['对比前5天均值(总注册)']= shuju['注册']-be5_data['注册']
all_ip['对比前7天均值(总注册)']= shuju['注册']-be7_data['注册']

all_ip['对比昨天(总开户)']=shuju['发送IP']-be_data['总IP']
all_ip['对比前3天均值(总开户)']= shuju['开户']-be3_data['开户']
all_ip['对比前5天均值(总开户)']= shuju['开户']-be5_data['开户']
all_ip['对比前7天均值(总开户)']= shuju['开户']-be7_data['开户']
# 纵向连接表格
shuju=shuju.join(all_ip,rsuffix='_2')
shuju.drop(['日期_2','人员2_2','发送IP_2'],inplace=True,axis=1)

C:\ProgramData\anaconda3\envs\schedule\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\User\AppData\Local\Temp\ipykernel_11320\809906320.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  be_data.sort_index(inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_11320\809906320.py:88: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  be3_data = be3_data.groupby('人员').mean()
C:\Users\User\AppData\Local\Temp\ipykernel_11320\809906320.py:90: FutureWarning: The 

In [6]:
shuju

,日期,人员2,发送IP,接受IP,注册,注册率,开户,转化率,当日注册并开户,当日注册激活率,对比昨天(总IP),对比前3天均值(总IP),对比前5天均值(总IP),对比前7天均值(总IP),对比昨天(总注册),对比前3天均值(总注册),对比前5天均值(总注册),对比前7天均值(总注册),对比昨天(总开户),对比前3天均值(总开户),对比前5天均值(总开户),对比前7天均值(总开户)
人员,,,,,,,,,,,,,,,,,,,,,,
Aber,2023-08-07,aber,7325,562,49,0.668942,13.0,26.530612,9.0,18.367347,-2150.0,-1503.000000,-1483.6,-768.142857,-2150.0,7.000000,8.4,9.000000,-2150.0,-3.666667,-1.8,-0.857143
Ben,2023-08-07,ben,2565,298,17,0.662768,6.0,35.294118,3.0,17.647059,-433.0,-144.333333,-793.8,-2063.714286,-433.0,7.333333,5.8,4.285714,-433.0,3.000000,3.4,3.285714
DK,2023-08-07,dk,1525,2160,52,3.409836,5.0,9.615385,5.0,9.615385,149.0,83.666667,57.8,47.857143,149.0,21.000000,21.4,20.571429,149.0,-3.000000,-3.8,-4.714286
Hugo,2023-08-07,hugo,103,0,16,15.533981,5.0,31.250000,2.0,12.500000,-83.0,-35.000000,-225.2,-446.000000,-83.0,5.333333,1.8,-0.285714,-83.0,0.000000,-1.6,-1.714286
Martin,2023-08-07,martin,3445,1072,81,2.351234,24.0,29.629630,18.0,22.222222,352.0,181.666667,285.6,620.714286,352.0,10.000000,20.8,27.714286,352.0,4.000000,6.6,8.571429
Max,2023-08-07,max,273,225,9,3.296703,NaN,NaN,NaN,NaN,-8.0,-7.333333,-5.0,-5.428571,-8.0,3.333333,3.2,3.571429,-8.0,NaN,NaN,NaN
Paddy,2023-08-07,paddy,0,51,3,inf,2.0,66.666667,2.0,66.666667,0.0,0.000000,0.0,-59.285714,0.0,1.666667,1.2,1.714286,0.0,1.333333,1.4,1.428571
Tony,2023-08-07,tony,568,1136,15,2.640845,3.0,20.000000,3.0,20.000000,-130.0,-51.000000,-69.6,-71.285714,-130.0,-0.333333,0.4,1.857143,-130.0,-2.000000,-2.2,-2.714286
Zed,2023-08-07,zed,4853,329,22,0.453328,8.0,36.363636,8.0,36.363636,623.0,260.000000,422.4,610.857143,623.0,9.666667,12.0,13.000000,623.0,1.333333,2.6,3.714286
